In [1]:
import os

os.chdir("../")
print(os.getcwd())

c:\Users\risko\Desktop\Prompt-Classification


In [ ]:
import random

import pandas as pd

random.seed(42)

In [3]:
results = pd.read_csv("reports/model_metrics.csv")

In [5]:
results_sorted = results.sort_values(by='latency')
results_sorted

,model,f1,accuracy,train_accuracy,recall,precision,cost,latency,date
19,fastText_finance_fasttext,94.34,94.30,94.49,93.57,95.12,0.0,1.169094e+04,2025-01-04 19:39:17.946764
12,fastText_healthcare_fasttext,98.61,98.60,98.67,98.66,98.56,0.0,1.545161e+04,2025-01-04 19:33:07.711438
4,fastText_law_fasttext,98.81,98.81,98.78,98.57,99.04,0.0,2.149092e+04,2025-01-04 19:22:40.063981
3,fastText_law_fasttext,98.79,98.80,98.77,98.59,98.99,0.0,2.187053e+04,2025-01-04 19:18:42.687004
0,fastText_law_fasttext,98.77,98.78,98.74,98.54,98.99,0.0,2.288228e+04,2025-01-04 19:09:35.416522
5,fastText_law_fasttext,98.77,98.79,98.75,98.56,98.99,0.0,2.294374e+04,2025-01-04 19:23:45.728613
14,XGBoost_healthcare_mini,99.90,99.90,99.99,99.88,99.93,0.0,3.172549e+05,2025-01-04 19:34:24.172344
7,XGBoost_law_mini,99.86,99.86,100.00,99.87,99.85,0.0,3.177272e+05,2025-01-04 19:25:12.213906
18,XGBoost_healthcare_baai,99.89,99.89,99.99,99.90,99.88,0.0,3.203994e+05,2025-01-04 19:39:12.884218
11,XGBoost_law_baai,99.73,99.74,100.00,99.67,99.80,0.0,3.210361e+05,2025-01-04 19:33:00.493060


In [8]:
results_sorted = results.sort_values(by='precision', ascending=False)
results_sorted

,model,f1,accuracy,train_accuracy,recall,precision,cost,latency,date
14,XGBoost_healthcare_mini,99.90,99.90,99.99,99.88,99.93,0.0,3.172549e+05,2025-01-04 19:34:24.172344
13,SVM_healthcare_mini,99.78,99.77,99.78,99.68,99.88,0.0,5.829084e+05,2025-01-04 19:34:18.048519
18,XGBoost_healthcare_baai,99.89,99.89,99.99,99.90,99.88,0.0,3.203994e+05,2025-01-04 19:39:12.884218
16,XGBoost_healthcare_tfidf,99.06,99.06,99.17,98.26,99.87,0.0,4.751795e+06,2025-01-04 19:38:18.472055
6,SVM_law_mini,99.84,99.84,99.93,99.80,99.87,0.0,5.956841e+05,2025-01-04 19:25:06.609748
7,XGBoost_law_mini,99.86,99.86,100.00,99.87,99.85,0.0,3.177272e+05,2025-01-04 19:25:12.213906
11,XGBoost_law_baai,99.73,99.74,100.00,99.67,99.80,0.0,3.210361e+05,2025-01-04 19:33:00.493060
9,XGBoost_law_tfidf,98.72,98.75,99.03,97.73,99.74,0.0,6.985026e+06,2025-01-04 19:31:31.535912
17,SVM_healthcare_baai,99.51,99.51,99.59,99.30,99.73,0.0,4.535523e+05,2025-01-04 19:39:06.391567
15,SVM_healthcare_tfidf,99.65,99.65,99.90,99.58,99.73,0.0,2.274634e+06,2025-01-04 19:37:32.629691


In [9]:
results_sorted = results.sort_values(by='accuracy', ascending=False)
results_sorted

,model,f1,accuracy,train_accuracy,recall,precision,cost,latency,date
14,XGBoost_healthcare_mini,99.90,99.90,99.99,99.88,99.93,0.0,3.172549e+05,2025-01-04 19:34:24.172344
18,XGBoost_healthcare_baai,99.89,99.89,99.99,99.90,99.88,0.0,3.203994e+05,2025-01-04 19:39:12.884218
7,XGBoost_law_mini,99.86,99.86,100.00,99.87,99.85,0.0,3.177272e+05,2025-01-04 19:25:12.213906
6,SVM_law_mini,99.84,99.84,99.93,99.80,99.87,0.0,5.956841e+05,2025-01-04 19:25:06.609748
13,SVM_healthcare_mini,99.78,99.77,99.78,99.68,99.88,0.0,5.829084e+05,2025-01-04 19:34:18.048519
11,XGBoost_law_baai,99.73,99.74,100.00,99.67,99.80,0.0,3.210361e+05,2025-01-04 19:33:00.493060
10,SVM_law_baai,99.71,99.71,99.81,99.75,99.67,0.0,4.640635e+05,2025-01-04 19:32:53.928585
15,SVM_healthcare_tfidf,99.65,99.65,99.90,99.58,99.73,0.0,2.274634e+06,2025-01-04 19:37:32.629691
1,SVM_law_tfidf,99.62,99.62,99.92,99.57,99.67,0.0,3.163521e+06,2025-01-04 19:14:42.471176
8,SVM_law_tfidf,99.56,99.56,99.92,99.52,99.60,0.0,3.335976e+06,2025-01-04 19:30:22.999070
